In [ ]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:
import pandas as pd
import re
df=pd.read_csv("stands.csv")
grads={"A":5,"B":4,"C":3,"D":2,"E":1,"∅":0,"?":None,"\xa0∅":0,'∞':5,"*":5,"C*":3,"B*":4,"D*":2,"A*":5}
gg="ABCDE∅"
def partfinder(s):
  if pd.isnull(s):
    return s
  elif "SO" in s.split(":")[0]:
    return 6
  elif "SC" in s.split(":")[0]:
    return 3
  elif "DU" in s.split(":")[0]:
    return 4
  elif "VA" in s.split(":")[0]:
    return 5
  elif "SBR" in s.split(":")[0]:
    return 7
  elif "JJL" in s.split(":")[0]:
    return 8
def chapterfinder(x):
  s=None
  if not(pd.isnull(x)):
    s=re.search(r'\d+',x)
  if s:
    return int(s.group(0))
  else:
    return None
df["Power"]=df["Power"].map(lambda x: grads[x] if not(pd.isnull(x)) else x)
df["Speed"]=df["Speed"].map(lambda x: grads[x] if not(pd.isnull(x)) else x)
df["Precision"]=df["Precision"].map(lambda x: grads[x] if not(pd.isnull(x)) else x)
df["Endurance"]=df["Endurance"].map(lambda x: grads[x] if not(pd.isnull(x)) else x)
df["Radius"]=df["Radius"].map(lambda x: grads[x] if not(pd.isnull(x)) else x)
df["Potential"]=df["Potential"].map(lambda x: grads[x] if not(pd.isnull(x)) else x)
df["Part"]=df["Manga"].map(lambda x: partfinder(x))
cols=['Power', 'Speed', 'Radius', 'Endurance',
       'Precision', 'Potential']
df["Chapter"]=df["Manga"].map(lambda x: chapterfinder(x))

In [ ]:

from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
data = df.set_index('Stand')[cols].transpose()

# external CSS stylesheets
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__,external_stylesheets=external_stylesheets)
colors = [
    "#bcb88a",
    "#ffb961",
    "#bdb76b",
    "#35170c",
    "#ff8b0f"
]
# df.groupby()

dff = data.columns[0]
fig = px.line_polar(data, r=dff, theta=cols, line_close=True,color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",)
fig.update_traces(fill='toself')

dsuns = dict(
    character=df["Stand"].tolist()+df["Part"].unique().tolist(),
    parent=df["Part"].tolist()+["" for i in range(df["Part"].unique().size)])
dsuns["value"]=[1 for i in range(len(dsuns["character"]))]
dsuns["color"]=[i%5 for i in range(len(dsuns["character"]))]
sunsb = px.sunburst(
    dsuns,
    names='character',
    parents='parent',
    values='value',
    #color="color",
    color_continuous_scale=colors
)
bardf=df[df["Power"]>4]
bars=px.bar(bardf, y="Stand", x=cols[0],orientation='h')
hist=px.histogram(df,x="Chapter")
# адаптивность диаграмм - встраивание в окно
style_dashboard={
          "height":'500px',
          "width":'100%',
          "display":"flex",
          "flex-direction":"column",
          "flex-grow":"0",
          "background-color":"#c3d4e0"
      }

app.layout = html.Div([
    html.H1(children='Jojo Stands', style={'textAlign':'center'}),
    html.Div([
    html.Div([
      html.H2(children='Best stands by parametr', style={'textAlign':'center'}),
      dcc.Dropdown(cols, cols[0], id='parameter',style={"height":"50px"}),
      dcc.Dropdown(["Up","Down"], "Up", id='arrow',style={"height":"50px"}),
      dcc.Graph(id='bar-chart',figure = bars, style={"height":'1000px'}),
    ],style={"height":'100%',
          "width":'100%',"border":"1px solid blue",
          "border-radius":"10px"}),
    html.Div([
        html.H2(children='Stand Map', style={'textAlign':'center'}),
        dcc.Dropdown(df["Stand"].unique(), 'Metallica', id='dropdown-selection',style={"height":"50px"}),
        html.Div(id = 'text-output', style={"height":"50px"}),
        dcc.Graph(id='graph-content', figure = fig, style=style_dashboard),
        dcc.Graph(id='sun-chart',figure = sunsb, style=style_dashboard)],style={"height":'100%',"width":'100%',"border":"1px solid blue","border-radius":"10px"})
    ],style={"height":'100%',
          "width":'100%',
          "display":"flex",
          "flex-direction":"row",
          "flex-grow":"0",
          }),
    html.H2(children='First appearance histogram', style={'textAlign':'center'}),
    dcc.Slider(10, 200, 10,
               value=20,
               id='bins-slider'
    ),
    dcc.Graph(id='hist-chart',figure = hist)
])


@callback(
    Output('text-output', 'children'),
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    ind=df[df["Stand"]==value].index.tolist()[0]
    dff=data.columns[ind]
    fig = px.line_polar(data, r=dff, theta=cols, line_close=True)
    fig.update_traces(fill='toself')
    text="Stand User - "+df[df["Stand"]==value]["User"]
    return text,fig

@callback(
    Output('bar-chart', 'figure'),
    Input('parameter', 'value'),
    Input('arrow', 'value')
)
def update_bar(vp,va):
    if va=="Up":
      bardf=df[df[vp]>4]
    else:
      bardf=df[df[vp]<2]
    bars=px.bar(bardf, y="Stand", x=vp,orientation='h')
    return bars

@callback(
    Output('hist-chart', 'figure'),
    Input('bins-slider', 'value')
)
def update_hist(value):
    hist=px.histogram(df,x="Chapter",nbins=value)
    return hist

if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>